### Analysing how topics are distributed within single full document

In [1]:
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import numpy as np
import os
import pickle
import pandas as pd
import nltk

from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords

In [3]:
import sys
sys.path.append("../")
from src import puhti_files

In [4]:
SEED = 42
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

In [5]:
training_sets = []
tp_models = []
for i in range(8):
    with open('../src/Training_sets/training_set'+str(i)+'.pickle', 'rb') as dataset:
        dataseti = pickle.load(dataset)
        training_sets.append(dataseti)
    with open('../src/tp_models/tp_model'+str(i)+'.pickle', 'rb') as tpmodel:
        tpmodeli = pickle.load(tpmodel)
        tp_models.append(tpmodeli)

In [6]:
models = []
for i in range(8):
    ctm = ZeroShotTM(bow_size=len(tp_models[i].vocab), contextual_size=768, n_components=10, num_epochs=20)
    ctm.load("../src/ctm_models"+str(i)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                          epoch=19)
    models.append(ctm)

/users/tturpein/.local/lib/python3.10/site-packages/contextualized_topic_models/models/ctm.py:669: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(
/users/tturpein/.local/lib/python3.10/site-packages/contextualized_topic_models/models/ctm.py:669: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(
/users/tturpein/.local/lib/python3.10/site-packages/contextualized_topic_models/models/ctm.py:669: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(
/users/tturpein/.local/lib/python3.10/site-packages/contextualized_topic_models/models/ctm.py:669: Warning: This is an experimental f

In [7]:
dict_dec = {}
count = 0
for i in range(1720,1800,10):
    dict_dec[i] = count
    count +=1
dict_dec

{1720: 0, 1730: 1, 1740: 2, 1750: 3, 1760: 4, 1770: 5, 1780: 6, 1790: 7}

In [8]:
df = puhti_files.genre_data_to_pandas(data="train", add_labels=True, merge_ecco=True, better_subcat_names=True)

Read in dataset train.csv. Set param 'data' to 'dev', 'test' or 'train' if you want another dataset.


In [9]:
### Filter by decade

df['publication_year'] = pd.to_numeric(df['publication_year'], errors='coerce')
df['decade'] = (df['publication_year'] // 10) * 10
df = df[df["decade"]==1720]
len(df)

1102

In [10]:
### Sample data
df = df[0:20].copy()

In [11]:
df["text"] = df["document_id"].apply(lambda x: puhti_files.read_text_file(x))

In [12]:
### Chunk documents

def split_text(text, num_words):
    words = text.split()
    chunks = [' '.join(words[i:i + num_words]) for i in range(0, len(words), num_words)]
    return chunks

num_words = 128
split_data = []
for _, row in df.iterrows():
    doc_id = row['document_id']
    text = row['text']
    chunks = split_text(text, num_words)
    
    for chunk in chunks:
        split_data.append({'document_id': doc_id, 'text': chunk})

split_df = pd.DataFrame(split_data)
len(split_df)

21887

In [13]:
from nltk.corpus import stopwords as stop_words
nltk.download('stopwords')
stopwords = list(stop_words.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /users/tturpein/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
sp = WhiteSpacePreprocessingStopwords(list(split_df["text"]), stopwords_list=stopwords, vocabulary_size=2000)
preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()

In [15]:
print(len(split_df))
print(len(preprocessed_documents))
print(len(unpreprocessed_corpus))

21887
21883
21883


In [16]:
index = dict_dec[1720]
model = models[index]
tp = tp_models[index]

In [17]:
model.USE_CUDA = True

In [18]:
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

Batches:   0%|          | 0/110 [00:00<?, ?it/s]

In [50]:
#import os
#os.environ["TOKENIZERS_PARALLELISM"] = "false"

#predictions = model.get_predicted_topics(dataset=training_dataset, n_samples=20)
#len(predictions)

In [51]:
#predictions_df = pd.DataFrame(predictions, columns=["Topic Prediction"])

In [52]:
#Resetting the index in case of dropped rows
#predictions_df = predictions_df.set_index(pd.Index(retained_indices))

In [53]:
#Combine to other data
#split_df = pd.concat([split_df, predictions_df], axis=1)
#split_df.shape

In [54]:
#split_df.columns

In [55]:
#split_df["Topic Prediction"].unique()

In [ ]:
#PREDS END

In [ ]:
### CHUNK AND DOC PREDS AT THE SAME TIME:

In [44]:
### Prediction per document_id
### Topic predictions as percentages for each topic for each document chunk
topics_predictions = model.get_thetas(training_dataset, n_samples=20)
topics_predictions.shape

/users/tturpein/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning:

This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

  0%|          | 0/342 [00:00<?, ?it/s]/users/tturpein/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning:

This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.

 94%|█████████▍| 321/342 [00:07<00:00, 269.55it/s]/CSC_

/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fbb38565600>

/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fb83824d960>

/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fbb38565600>

/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fb83824d960>

/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fbb38565600>

/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/multiprocessing/popen_fork.py:66: ResourceWarning:

Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7fb83824d960>

/CSC_CONTAINER/miniconda/envs/env1/lib/p

(21883, 10)

In [57]:
topic_number = np.argmax(topics_predictions[0])
topic_number

1

In [62]:
best_topics = np.argmax(topics_predictions, axis=1)
len(best_topics)

21883

In [60]:
topics_predictions

(10,)

In [63]:
### Adding the predictions to a dataframe
topics = 10
topic_columns = [f"topic_{i}_prob" for i in range(0, topics)]
predictions_df = pd.DataFrame(topics_predictions, columns=topic_columns)
predictions_df['Topic Prediction'] = best_topics

In [64]:
predictions_df

,topic_0_prob,topic_1_prob,topic_2_prob,topic_3_prob,topic_4_prob,topic_5_prob,topic_6_prob,topic_7_prob,topic_8_prob,topic_9_prob,Best_Topic
0,0.074155,0.370205,0.039106,0.019023,0.051827,0.016596,0.051501,0.279886,0.033186,0.064514,1
1,0.022129,0.448448,0.028700,0.022599,0.020366,0.023867,0.029967,0.343303,0.028254,0.032365,1
2,0.052701,0.423831,0.045592,0.018765,0.069527,0.052151,0.043433,0.226795,0.028842,0.038364,1
3,0.036131,0.412810,0.043909,0.032600,0.034011,0.025903,0.052673,0.287231,0.020439,0.054293,1
4,0.054280,0.344125,0.034888,0.037550,0.017348,0.042109,0.028373,0.392404,0.023930,0.024993,7
...,...,...,...,...,...,...,...,...,...,...,...
21878,0.073932,0.170234,0.150378,0.137557,0.081474,0.160151,0.028988,0.115808,0.036715,0.044764,1
21879,0.040137,0.363155,0.029256,0.107497,0.017730,0.037839,0.027540,0.271268,0.037112,0.068467,1
21880,0.049774,0.099416,0.038614,0.211051,0.050759,0.143798,0.075344,0.076975,0.097376,0.156894,3
21881,0.090135,0.096201,0.098020,0.103297,0.079046,0.261313,0.067019,0.097025,0.042909,0.065033,5


In [65]:
#Resetting the index and combining
predictions_df = predictions_df.set_index(pd.Index(retained_indices))
split_df = pd.concat([split_df, predictions_df], axis=1)
split_df.shape

(21887, 24)

In [66]:
split_df

,document_id,text,Topic Prediction,topic_0_prob,topic_1_prob,topic_2_prob,topic_3_prob,topic_4_prob,topic_5_prob,topic_6_prob,...,topic_1_prob,topic_2_prob,topic_3_prob,topic_4_prob,topic_5_prob,topic_6_prob,topic_7_prob,topic_8_prob,topic_9_prob,Best_Topic
0,0227200600,JUST INU S De HISTORIIS PHILIP PlC IS ET TOTIU...,7.0,0.074155,0.370205,0.039106,0.019023,0.051827,0.016596,0.051501,...,0.370205,0.039106,0.019023,0.051827,0.016596,0.051501,0.279886,0.033186,0.064514,1.0
1,0227200600,"ju. it. Ludovici Magni prodit iterum in lucem,...",1.0,0.022129,0.448448,0.028700,0.022599,0.020366,0.023867,0.029967,...,0.448448,0.028700,0.022599,0.020366,0.023867,0.029967,0.343303,0.028254,0.032365,1.0
2,0227200600,"tibi fuis & virtutibua multrm procdeffe, &viti...",1.0,0.052701,0.423831,0.045592,0.018765,0.069527,0.052151,0.043433,...,0.423831,0.045592,0.018765,0.069527,0.052151,0.043433,0.226795,0.028842,0.038364,1.0
3,0227200600,"& auferas in relus, turn fecundis, turn adverf...",1.0,0.036131,0.412810,0.043909,0.032600,0.034011,0.025903,0.052673,...,0.412810,0.043909,0.032600,0.034011,0.025903,0.052673,0.287231,0.020439,0.054293,1.0
4,0227200600,jura dabit popalis; tu eruditis L ille praliis...,1.0,0.054280,0.344125,0.034888,0.037550,0.017348,0.042109,0.028373,...,0.344125,0.034888,0.037550,0.017348,0.042109,0.028373,0.392404,0.023930,0.024993,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21882,1395501300,"has; cws by aurMis admir'd, Alas!i your fruitl...",1.0,0.073932,0.170234,0.150378,0.137557,0.081474,0.160151,0.028988,...,0.170234,0.150378,0.137557,0.081474,0.160151,0.028988,0.115808,0.036715,0.044764,1.0
21883,1395501300,"Ida's Combat now renew'd, V V .:Each Goddess t...",1.0,0.040137,0.363155,0.029256,0.107497,0.017730,0.037839,0.027540,...,0.363155,0.029256,0.107497,0.017730,0.037839,0.027540,0.271268,0.037112,0.068467,1.0
21884,1395501300,~Found no Return but cold Disdain. Yet flill w...,3.0,0.049774,0.099416,0.038614,0.211051,0.050759,0.143798,0.075344,...,0.099416,0.038614,0.211051,0.050759,0.143798,0.075344,0.076975,0.097376,0.156894,3.0
21885,1395501300,I of them for ever be. Nay: could I gaid the R...,5.0,0.090135,0.096201,0.098020,0.103297,0.079046,0.261313,0.067019,...,0.096201,0.098020,0.103297,0.079046,0.261313,0.067019,0.097025,0.042909,0.065033,5.0


In [71]:
mean_prob_df = split_df.groupby('document_id')[topic_columns].mean()
mean_prob_df['Document Topic Prediction'] = mean_prob_df.idxmax(axis=1).str.replace('_prob', '').str.replace('topic_', '').astype(int)
mean_prob_df = mean_prob_df.reset_index()
print(len(mean_prob_df))
mean_prob_df.head(3)

20


,document_id,topic_0_prob,topic_0_prob,topic_1_prob,topic_1_prob,topic_2_prob,topic_2_prob,topic_3_prob,topic_3_prob,topic_4_prob,...,topic_5_prob,topic_6_prob,topic_6_prob,topic_7_prob,topic_7_prob,topic_8_prob,topic_8_prob,topic_9_prob,topic_9_prob,Document Topic Prediction
0,0009700110,0.100040,0.100040,0.147545,0.147545,0.151907,0.151907,0.097650,0.097650,0.07349,...,0.114984,0.054132,0.054132,0.130270,0.130270,0.053405,0.053405,0.076576,0.076576,2
1,0013600101,0.112090,0.112090,0.183076,0.183076,0.127672,0.127672,0.064476,0.064476,0.07305,...,0.068198,0.054426,0.054426,0.167792,0.167792,0.047161,0.047161,0.102060,0.102060,1
2,0081300106,0.084004,0.084004,0.209612,0.209612,0.087527,0.087527,0.089011,0.089011,0.06529,...,0.084677,0.057845,0.057845,0.176270,0.176270,0.054375,0.054375,0.091388,0.091388,1


In [80]:
split_df = split_df.merge(mean_prob_df[['document_id', 'Document Topic Prediction']], on='document_id', how='left')

In [81]:
split_df.head(3)

,document_id,text,Topic Prediction,topic_0_prob,topic_1_prob,topic_2_prob,topic_3_prob,topic_4_prob,topic_5_prob,topic_6_prob,...,topic_2_prob,topic_3_prob,topic_4_prob,topic_5_prob,topic_6_prob,topic_7_prob,topic_8_prob,topic_9_prob,Best_Topic,Document Topic Prediction
0,0227200600,JUST INU S De HISTORIIS PHILIP PlC IS ET TOTIU...,7.0,0.074155,0.370205,0.039106,0.019023,0.051827,0.016596,0.051501,...,0.039106,0.019023,0.051827,0.016596,0.051501,0.279886,0.033186,0.064514,1.0,1
1,0227200600,"ju. it. Ludovici Magni prodit iterum in lucem,...",1.0,0.022129,0.448448,0.028700,0.022599,0.020366,0.023867,0.029967,...,0.028700,0.022599,0.020366,0.023867,0.029967,0.343303,0.028254,0.032365,1.0,1
2,0227200600,"tibi fuis & virtutibua multrm procdeffe, &viti...",1.0,0.052701,0.423831,0.045592,0.018765,0.069527,0.052151,0.043433,...,0.045592,0.018765,0.069527,0.052151,0.043433,0.226795,0.028842,0.038364,1.0,1


In [82]:
df_topic_count = split_df.groupby(['document_id', 'Topic Prediction']).size().reset_index(name='Count')

In [83]:
df_topic_count

,document_id,Topic Prediction,Count
0,0009700110,0.0,29
1,0009700110,1.0,77
2,0009700110,2.0,95
3,0009700110,3.0,25
4,0009700110,4.0,9
...,...,...,...
164,1678300300,5.0,15
165,1678300300,6.0,2
166,1678300300,7.0,3
167,1678300300,8.0,2


In [84]:
df_total_count = df_topic_count.groupby('document_id')['Count'].sum().reset_index(name='Total')
df_total_count

,document_id,Total
0,0009700110,335
1,0013600101,437
2,0081300106,528
3,0081300108,396
4,0089000603,173
5,0227200600,1659
6,0345601400,195
7,0370900702,692
8,0439700100,1049
9,0489400101,13316


In [85]:
df_merged = df_topic_count.merge(df_total_count, on='document_id')
df_merged['Proportion'] = df_merged['Count'] / df_merged['Total']
df_merged

,document_id,Topic Prediction,Count,Total,Proportion
0,0009700110,0.0,29,335,0.086567
1,0009700110,1.0,77,335,0.229851
2,0009700110,2.0,95,335,0.283582
3,0009700110,3.0,25,335,0.074627
4,0009700110,4.0,9,335,0.026866
...,...,...,...,...,...
164,1678300300,5.0,15,31,0.483871
165,1678300300,6.0,2,31,0.064516
166,1678300300,7.0,3,31,0.096774
167,1678300300,8.0,2,31,0.064516


In [86]:
df_merged["Topic Prediction"] = df_merged["Topic Prediction"].astype(str)
df_merged["Topic Prediction"] = df_merged["Topic Prediction"].apply(lambda x: x.strip(".")[0])
df_merged.dtypes

document_id          object
Topic Prediction     object
Count                 int64
Total                 int64
Proportion          float64
dtype: object

In [36]:
df_merged

,document_id,Topic Prediction,Count,Total,Proportion
0,0009700110,0,29,335,0.086567
1,0009700110,1,77,335,0.229851
2,0009700110,2,95,335,0.283582
3,0009700110,3,25,335,0.074627
4,0009700110,4,9,335,0.026866
...,...,...,...,...,...
164,1678300300,5,15,31,0.483871
165,1678300300,6,2,31,0.064516
166,1678300300,7,3,31,0.096774
167,1678300300,8,2,31,0.064516


In [38]:
df_merged = df_merged.sort_values(by=['document_id', 'Topic Prediction'])
df_merged

,document_id,Topic Prediction,Count,Total,Proportion
0,0009700110,0,29,335,0.086567
1,0009700110,1,77,335,0.229851
2,0009700110,2,95,335,0.283582
3,0009700110,3,25,335,0.074627
4,0009700110,4,9,335,0.026866
...,...,...,...,...,...
164,1678300300,5,15,31,0.483871
165,1678300300,6,2,31,0.064516
166,1678300300,7,3,31,0.096774
167,1678300300,8,2,31,0.064516


In [87]:
df_merged = df_merged.merge(mean_prob_df[['document_id', 'Document Topic Prediction']], on='document_id', how='left')
df_merged

,document_id,Topic Prediction,Count,Total,Proportion,Document Topic Prediction
0,0009700110,0,29,335,0.086567,2
1,0009700110,1,77,335,0.229851,2
2,0009700110,2,95,335,0.283582,2
3,0009700110,3,25,335,0.074627,2
4,0009700110,4,9,335,0.026866,2
...,...,...,...,...,...,...
164,1678300300,5,15,31,0.483871,5
165,1678300300,6,2,31,0.064516,5
166,1678300300,7,3,31,0.096774,5
167,1678300300,8,2,31,0.064516,5


In [88]:
df_merged['Document_and_Prediction'] = df_merged['document_id'].astype(str) + ' - Prediction: ' + df_merged['Document Topic Prediction'].astype(str)

In [40]:
topic_order = sorted(df_merged['Topic Prediction'].unique())

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [97]:
import plotly.express as px

fig = px.bar(
    df_merged,
    x='document_id',
    y='Proportion',
    color='Topic Prediction',
    text='Topic Prediction',
    title='Topic Predictions Distribution per Document',
    height=600,
    category_orders={'Topic Prediction': topic_order},
)

fig.update_traces(textposition='inside')
fig.update_layout(
    showlegend=False,
    xaxis_title="Full Document Prediction"
)

doc_ids = df_merged['document_id'].unique()
doc_preds = df_merged.drop_duplicates(subset='document_id')['Document Topic Prediction'].values
custom_tick_labels = [f"{pred}" for pred in doc_preds]

fig.update_xaxes(
    tickmode='array',
    tickvals=doc_ids,
    ticktext=custom_tick_labels,
)


fig.show()

In [98]:
import plotly.graph_objs as go

# Pivot the dataframe to have document_id as rows, Topic Prediction as columns, and Proportion as values
heatmap_data = df_merged.pivot(index='document_id', columns='Topic Prediction', values='Proportion')
heatmap_data = heatmap_data.fillna(0)  # Fill NaN values with 0

fig = go.Figure(go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Viridis',
))

fig.update_layout(
    title='Topic Predictions Distribution per Document',
    xaxis_title='Topic Prediction',
    yaxis_title='Document ID',
    height=600
)

fig.show()